In [1]:
print("dd")

dd


In [3]:
import os
import shutil
import random
from glob import glob

# 1. 경로 설정
# 원본 데이터가 있는 경로
source_root = r'C:\Dev\KorailWheel\data\train' 
source_img_dir = os.path.join(source_root, 'images')
source_lbl_dir = os.path.join(source_root, 'labels')

# 결과가 저장될 경로 (새로 생성됨)
output_root = r'C:\Dev\KorailWheel\data\split_dataset'

# 2. 비율 설정 (7 : 1.5 : 1.5)
train_ratio = 0.7
valid_ratio = 0.15
test_ratio = 0.15

# 3. 결과 디렉토리 생성 함수
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

# 폴더 구조 생성: images/train, labels/train 등
for split in ['train', 'valid', 'test']:
    create_dir(os.path.join(output_root, 'images', split))
    create_dir(os.path.join(output_root, 'labels', split))

# 4. 파일 리스트 로드 및 셔플
# 이미지 확장자 (필요시 추가)
image_exts = ['.jpg', '.jpeg', '.png', '.bmp']
files = [f for f in os.listdir(source_img_dir) if os.path.splitext(f)[1].lower() in image_exts]

# 재현성을 위해 시드 고정 (선택 사항)
random.seed(42)
random.shuffle(files)

# 5. 분할 지점 계산
total_count = len(files)
train_idx = int(total_count * train_ratio)
valid_idx = train_idx + int(total_count * valid_ratio)

# 리스트 슬라이싱
train_files = files[:train_idx]
valid_files = files[train_idx:valid_idx]
test_files = files[valid_idx:]

print(f"총 파일 수: {total_count}")
print(f"Train: {len(train_files)} / Valid: {len(valid_files)} / Test: {len(test_files)}")
print("-" * 30)

# 6. 파일 복사 실행 함수
def copy_files(file_list, split_name):
    print(f"📂 {split_name} 데이터 복사 중...")
    for file_name in file_list:
        # 파일명과 확장자 분리
        name, ext = os.path.splitext(file_name)
        
        # 소스 경로
        src_img = os.path.join(source_img_dir, file_name)
        src_lbl = os.path.join(source_lbl_dir, name + '.txt')
        
        # 타겟 경로
        dst_img = os.path.join(output_root, 'images', split_name, file_name)
        dst_lbl = os.path.join(output_root, 'labels', split_name, name + '.txt')
        
        # 이미지 복사
        shutil.copy2(src_img, dst_img)
        
        # 라벨 복사 (라벨이 존재하는 경우만)
        if os.path.exists(src_lbl):
            shutil.copy2(src_lbl, dst_lbl)

# 실행
copy_files(train_files, 'train')
copy_files(valid_files, 'valid')
copy_files(test_files, 'test')

print("-" * 30)
print(f"✅ 데이터셋 분할 완료! 저장 경로: {output_root}")

총 파일 수: 368
Train: 257 / Valid: 55 / Test: 56
------------------------------
📂 train 데이터 복사 중...
📂 valid 데이터 복사 중...
📂 test 데이터 복사 중...
------------------------------
✅ 데이터셋 분할 완료! 저장 경로: C:\Dev\KorailWheel\data\split_dataset


In [2]:
import os
import shutil
import random
from pathlib import Path
from tqdm import tqdm  # 진행상황 표시용 (없으면 pip install tqdm)

# ==========================================
# 1. 설정 (경로 확인 필수)
# ==========================================
BASE_DIR = Path("C:/Dev/KorailWheel/data")  # 프로젝트 루트 경로
SOURCE_DIR = BASE_DIR / "total"        # 원본 소스 폴더
DEST_DIR = BASE_DIR / "data"           # 분할된 데이터가 저장될 폴더

# 시드 고정 (재현성을 위해)
random.seed(42)

# ==========================================
# 2. 파일 목록 읽기 및 분류
# ==========================================
image_dir = SOURCE_DIR / "images"
label_dir = SOURCE_DIR / "labels"

# 이미지 파일 확장자 목록
valid_extensions = {".jpg", ".jpeg", ".png", ".bmp"}

# 전체 이미지 리스트 가져오기
all_images = [
    f for f in image_dir.iterdir() 
    if f.suffix.lower() in valid_extensions and f.is_file()
]

print(f"총 이미지 개수: {len(all_images)}장")

# 그룹 분류
# 'img'로 시작하는 파일 (고품질/오토라벨 보정본 추정) -> Valid/Test 후보
candidates = [img for img in all_images if img.name.lower().startswith("img")]

# 그 외 파일 (수동 라벨링 등) -> 무조건 Train
others = [img for img in all_images if not img.name.lower().startswith("img")]

print(f"- 'img'로 시작하는 파일(후보군): {len(candidates)}장")
print(f"- 그 외 파일(Train 확정): {len(others)}장")

# ==========================================
# 3. 데이터 분할 (Split)
# ==========================================
# 후보군이 최소 100장은 넘어야 함 (Valid 50 + Test 50)
if len(candidates) < 100:
    raise ValueError(f"오류: 'img'로 시작하는 파일이 100장 미만입니다. ({len(candidates)}장)")

# 랜덤 셔플
random.shuffle(candidates)

# 50장씩 뽑기
valid_set = candidates[:50]
test_set = candidates[50:100]

# 나머지는 모두 Train으로
train_from_candidates = candidates[100:]
train_set = train_from_candidates + others

print("\n[데이터셋 분할 결과]")
print(f"- Train : {len(train_set)}장 (나머지 img파일 + 그 외 파일)")
print(f"- Valid : {len(valid_set)}장 (img파일 중 랜덤 50장)")
print(f"- Test  : {len(test_set)}장 (img파일 중 랜덤 50장)")

# ==========================================
# 4. 파일 복사 실행
# ==========================================
def copy_files(file_list, split_name):
    # 저장할 경로 생성 (예: data/train/images)
    save_img_dir = DEST_DIR / split_name / "images"
    save_lbl_dir = DEST_DIR / split_name / "labels"
    
    save_img_dir.mkdir(parents=True, exist_ok=True)
    save_lbl_dir.mkdir(parents=True, exist_ok=True)
    
    print(f"\nProcessing {split_name} data...")
    
    for img_path in tqdm(file_list):
        # 1. 이미지 복사
        shutil.copy2(img_path, save_img_dir / img_path.name)
        
        # 2. 라벨 복사 (짝이 맞는 txt 파일 찾기)
        label_name = img_path.stem + ".txt"
        label_src = label_dir / label_name
        
        if label_src.exists():
            shutil.copy2(label_src, save_lbl_dir / label_name)
        else:
            # 라벨이 없는 경우 경고 (필요 시 주석 해제)
            # print(f"Warning: Label not found for {img_path.name}")
            pass

# 기존 data 폴더가 있다면 삭제 후 재생성 (깨끗한 상태 유지)
if DEST_DIR.exists():
    shutil.rmtree(DEST_DIR)
    print(f"\n기존 {DEST_DIR} 폴더를 삭제하고 새로 생성합니다.")

# 복사 수행
copy_files(train_set, "train")
copy_files(valid_set, "valid")
copy_files(test_set, "test")

print("\n✅ 모든 작업이 완료되었습니다! 'data' 폴더를 확인하세요.")

총 이미지 개수: 368장
- 'img'로 시작하는 파일(후보군): 268장
- 그 외 파일(Train 확정): 100장

[데이터셋 분할 결과]
- Train : 268장 (나머지 img파일 + 그 외 파일)
- Valid : 50장 (img파일 중 랜덤 50장)
- Test  : 50장 (img파일 중 랜덤 50장)

Processing train data...


100%|██████████| 268/268 [00:00<00:00, 272.84it/s]



Processing valid data...


100%|██████████| 50/50 [00:00<00:00, 268.89it/s]



Processing test data...


100%|██████████| 50/50 [00:00<00:00, 301.07it/s]


✅ 모든 작업이 완료되었습니다! 'data' 폴더를 확인하세요.


In [1]:
# build_patch_lib.py
from pathlib import Path
import cv2
import numpy as np
import pandas as pd

from config import ROOT

# -------------------------
# Settings (ONLY here)
# -------------------------
SRC_DIR = ROOT / "data" / "patch_all"          # images/, labels/
OUT_DIR = ROOT / "data" / "patch"              # train/valid/test 생성

SEED = 0
RATIO_TRAIN = 0.70
RATIO_VALID = 0.15
RATIO_TEST  = 0.15

# patch crop 옵션
MARGIN = 16                 # 인스턴스 bbox 바깥 여유 픽셀
MIN_MASK_AREA = 120         # 너무 작은 인스턴스(노이즈) 제거
APPROX_EPS = 2.0            # contour 폴리곤 단순화 정도(픽셀)

# 클래스 필터(필요시)
# None이면 모든 클래스 추출, 예: {0} 처럼 지정하면 해당 cls만 추출
KEEP_CLASSES = None

# 출력 포맷
IMG_EXT = ".png"            # crop 저장 확장자


# -------------------------
# Utils
# -------------------------
def _img_list(folder: Path):
    exts = {".jpg", ".jpeg", ".png", ".bmp", ".webp"}
    return sorted([p for p in folder.rglob("*") if p.suffix.lower() in exts])


def _read_yolo_seg_instances(label_path: Path):
    """
    YOLOv8-seg line: cls xc yc w h x1 y1 x2 y2 ... (normalized)
    return: list[(cls:int, poly_norm:(N,2) float32)]
    """
    if not label_path.exists():
        return []
    txt = label_path.read_text(encoding="utf-8").strip()
    if not txt:
        return []

    inst = []
    for line in txt.splitlines():
        p = line.strip().split()
        if len(p) < 6:
            continue
        cls = int(float(p[0]))
        coords = np.array(list(map(float, p[5:])), dtype=np.float32)
        if coords.size >= 6:
            poly = coords.reshape(-1, 2)
            inst.append((cls, poly))
    return inst


def _poly_to_mask(poly_norm: np.ndarray, w: int, h: int):
    m = np.zeros((h, w), dtype=np.uint8)
    pts = poly_norm.copy()
    pts[:, 0] = np.clip(pts[:, 0] * w, 0, w - 1)
    pts[:, 1] = np.clip(pts[:, 1] * h, 0, h - 1)
    pts = pts.astype(np.int32)
    if pts.shape[0] >= 3:
        cv2.fillPoly(m, [pts], 1)
    return m


def _crop_with_margin(img, mask01, margin):
    ys, xs = np.where(mask01 > 0)
    if ys.size == 0:
        return None
    y0, y1 = ys.min(), ys.max()
    x0, x1 = xs.min(), xs.max()

    H, W = mask01.shape
    x0 = max(0, int(x0) - margin)
    y0 = max(0, int(y0) - margin)
    x1 = min(W - 1, int(x1) + margin)
    y1 = min(H - 1, int(y1) + margin)

    crop_img = img[y0:y1+1, x0:x1+1].copy()
    crop_m   = mask01[y0:y1+1, x0:x1+1].copy()
    return crop_img, crop_m, x0, y0


def _mask_to_yolo_seg_line(mask01_crop, cls_id):
    """
    crop 마스크에서 contour 기반으로 (bbox + polygon) YOLOv8-seg 라인 생성
    return: str or None
    """
    m = (mask01_crop > 0).astype(np.uint8)
    cnts, _ = cv2.findContours(m, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not cnts:
        return None
    cnt = max(cnts, key=cv2.contourArea)
    if cv2.contourArea(cnt) < MIN_MASK_AREA:
        return None

    # polygon 단순화
    eps = APPROX_EPS
    approx = cv2.approxPolyDP(cnt, eps, True)
    if approx.shape[0] < 3:
        return None

    H, W = m.shape[:2]
    pts = approx.reshape(-1, 2).astype(np.float32)
    pts[:, 0] = np.clip(pts[:, 0], 0, W - 1)
    pts[:, 1] = np.clip(pts[:, 1], 0, H - 1)

    # bbox from contour
    x, y, bw, bh = cv2.boundingRect(cnt)
    xc = (x + bw / 2) / W
    yc = (y + bh / 2) / H
    bw_n = bw / W
    bh_n = bh / H

    # polygon normalized
    pts_n = pts.copy()
    pts_n[:, 0] = pts_n[:, 0] / W
    pts_n[:, 1] = pts_n[:, 1] / H

    # line
    vals = [f"{cls_id:d}", f"{xc:.6f}", f"{yc:.6f}", f"{bw_n:.6f}", f"{bh_n:.6f}"]
    for (px, py) in pts_n:
        vals.append(f"{px:.6f}")
        vals.append(f"{py:.6f}")
    return " ".join(vals)


def _make_splits(src_stems, seed, r_tr, r_va, r_te):
    rng = np.random.default_rng(seed)
    stems = np.array(sorted(list(src_stems)))
    rng.shuffle(stems)

    n = len(stems)
    n_tr = int(round(n * r_tr))
    n_va = int(round(n * r_va))
    n_te = n - n_tr - n_va

    tr = set(stems[:n_tr])
    va = set(stems[n_tr:n_tr + n_va])
    te = set(stems[n_tr + n_va:])

    return tr, va, te


def main():
    img_dir = SRC_DIR / "images"
    lab_dir = SRC_DIR / "labels"

    imgs = _img_list(img_dir)
    # "라벨 있는 원본"만 split 기준으로 쓰는 게 안정적
    labeled = []
    for p in imgs:
        if (lab_dir / f"{p.stem}.txt").exists():
            labeled.append(p)

    src_stems = {p.stem for p in labeled}
    tr_set, va_set, te_set = _make_splits(src_stems, SEED, RATIO_TRAIN, RATIO_VALID, RATIO_TEST)

    def _split_of(stem):
        if stem in tr_set: return "train"
        if stem in va_set: return "valid"
        return "test"

    # output dirs
    for sp in ["train", "valid", "test"]:
        (OUT_DIR / sp / "images").mkdir(parents=True, exist_ok=True)
        (OUT_DIR / sp / "labels").mkdir(parents=True, exist_ok=True)
        (OUT_DIR / sp / "masks").mkdir(parents=True, exist_ok=True)

    # save split mapping (src image 기준)
    split_rows = []
    for stem in sorted(src_stems):
        split_rows.append({"src_stem": stem, "split": _split_of(stem)})
    pd.DataFrame(split_rows).to_csv(OUT_DIR / "splits_src.csv", index=False, encoding="utf-8-sig")

    # extract instances
    manifest = []
    for img_path in labeled:
        stem = img_path.stem
        split = _split_of(stem)

        img = cv2.imread(str(img_path))
        if img is None:
            continue
        H, W = img.shape[:2]

        inst = _read_yolo_seg_instances(lab_dir / f"{stem}.txt")
        if KEEP_CLASSES is not None:
            inst = [(c, p) for (c, p) in inst if c in KEEP_CLASSES]

        for k, (cls_id, poly_norm) in enumerate(inst):
            mask01 = _poly_to_mask(poly_norm, W, H)
            area = int(mask01.sum())
            if area < MIN_MASK_AREA:
                continue

            cropped = _crop_with_margin(img, mask01, MARGIN)
            if cropped is None:
                continue
            crop_img, crop_m, x0, y0 = cropped

            yolo_line = _mask_to_yolo_seg_line(crop_m, cls_id)
            if yolo_line is None:
                continue

            out_stem = f"{stem}_i{k:02d}_c{cls_id}"
            out_img = OUT_DIR / split / "images" / f"{out_stem}{IMG_EXT}"
            out_lab = OUT_DIR / split / "labels" / f"{out_stem}.txt"
            out_msk = OUT_DIR / split / "masks"  / f"{out_stem}.png"

            cv2.imwrite(str(out_img), crop_img)
            cv2.imwrite(str(out_msk), (crop_m > 0).astype(np.uint8) * 255)
            out_lab.write_text(yolo_line + "\n", encoding="utf-8")

            manifest.append({
                "split": split,
                "src_image": str(img_path),
                "src_stem": stem,
                "inst_idx": k,
                "cls": cls_id,
                "area": area,
                "crop_x0": int(x0),
                "crop_y0": int(y0),
                "crop_w": int(crop_img.shape[1]),
                "crop_h": int(crop_img.shape[0]),
                "out_image": str(out_img),
                "out_mask": str(out_msk),
                "out_label": str(out_lab),
            })

    pd.DataFrame(manifest).to_csv(OUT_DIR / "manifest.csv", index=False, encoding="utf-8-sig")

    print("DONE")
    print("Saved splits:", OUT_DIR / "splits_src.csv")
    print("Saved manifest:", OUT_DIR / "manifest.csv")
    df = pd.DataFrame(manifest)
    if len(df):
        print(df.groupby("split").size())
    else:
        print("No instances extracted. Check labels/classes/min_area.")


if __name__ == "__main__":
    main()


DONE
Saved splits: C:\Dev\KorailWheel\data\patch\splits_src.csv
Saved manifest: C:\Dev\KorailWheel\data\patch\manifest.csv
split
test     25
train    85
valid    21
dtype: int64


In [ ]:
from pathlib import Path
base = Path(r"C:\Dev\KorailWheel\data\st1_roi")

NameError: name 'Path' is not defined

In [2]:
def _wheel_id_quick(p: Path):
    # 파일명 규칙에 맞게 수정: 기본은 첫 '_' 전까지
    s = p.stem
    return s.split("_")[0] if "_" in s else s

NameError: name 'Path' is not defined

In [ ]:
def check_leakage():
    
    sets = {}
    for sp in ["train", "valid", "test"]:
        imgs = _img_list(base / sp / "images")
        sets[sp] = set(_wheel_id_quick(p) for p in imgs)

    inter_tv = sets["train"] & sets["valid"]
    inter_tt = sets["train"] & sets["test"]
    inter_vt = sets["valid"] & sets["test"]

    print("train wheels:", len(sets["train"]))
    print("valid wheels:", len(sets["valid"]))
    print("test  wheels:", len(sets["test"]))
    print("overlap train-valid:", len(inter_tv))
    print("overlap train-test :", len(inter_tt))
    print("overlap valid-test :", len(inter_vt))
    if len(inter_tv)+len(inter_tt)+len(inter_vt) > 0:
        print("LEAKAGE: wheel_id overlap exists")
    else:
        print("OK: no wheel_id overlap")

# check_leakage()
